
```
███╗   ███╗██╗███╗   ██╗███████╗ ██████╗ ██████╗ ██╗      █████╗ ██████╗ 
████╗ ████║██║████╗  ██║██╔════╝██╔════╝██╔═══██╗██║     ██╔══██╗██╔══██╗
██╔████╔██║██║██╔██╗ ██║█████╗  ██║     ██║   ██║██║     ███████║██████╔╝
██║╚██╔╝██║██║██║╚██╗██║██╔══╝  ██║     ██║   ██║██║     ██╔══██║██╔══██╗
██║ ╚═╝ ██║██║██║ ╚████║███████╗╚██████╗╚██████╔╝███████╗██║  ██║██████╔╝
╚═╝     ╚═╝╚═╝╚═╝  ╚═══╝╚══════╝ ╚═════╝ ╚═════╝ ╚══════╝╚═╝  ╚═╝╚═════╝ 
```
**Run a Minecraft Server on Google Colab!**

---

The script below will run your server. You'll have to create a server first to be able to use it - don't worry, the scripts below will do the majority of the work for you. You might also want to change the default region to your region, check below.

In [ ]:
import os
import re
import json
import glob

# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Change directory to the Minecraft server folder on Google Drive
%cd "/content/drive/My Drive/Minecraft-server"
!ls #list the directory contents (to verify that working directory was changed)

# Import config file.
if os.path.isfile("colabconfig.json"):
  colabconfig = json.load(open("colabconfig.json"))
else:
  colabconfig = {"server_type": "generic"} # using default, if config doesn't exists.
  json.dump(colabconfig, open("colabconfig.json",'w'))


# Install OpenJDK 17
# !wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
# !sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk17 gets installed."
# !sudo apt-get install openjdk-17-jre-headless &>/dev/null && echo "Yay! Openjdk17 has been successfully installed." || echo "Failed to install OpenJdk17."

version = colabconfig["server_version"]
if colabconfig["server_version"] < "1.17" :
  !sudo apt-get purge openjdk* > /dev/null 2>&1
  !sudo apt-get install openjdk-8-jre-headless &>/dev/null && echo "Yay! Openjdk8 has been successfully installed." || echo "Failed to install OpenJdk8."
else:
  !sudo apt-get purge openjdk* > /dev/null 2>&1
  !sudo apt-get install openjdk-17-jre-headless &>/dev/null && echo "Yay! Openjdk17 has been successfully installed." || echo "Failed to install OpenJdk17."



#Perform java version check
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
if java_ver[0] == "17" :
  print("Esta utilizando JAVA 17 / JAVA 17 is working correctly.")
else:
  print("Esta utilizando una version inferior a JAVA 17: ", java_ver[0], ". You might experience reduced performance. Minecraft 1.17 and above might fail to launch.")


# Server jar names.
jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar', 'forge': 'forge.jar', 'vanilla': 'vanilla.jar', 'snapshot': 'snapshot.jar'}
jar_name = jar_list[colabconfig["server_type"]]

# Java arguments.
if colabconfig["server_type"] == "paper":
  server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
else:
  server_flags = "" # aiker's flags might negatively impact performance on non-paper servers.
memory_allocation = "-Xms8704M -Xmx8704M"

# Chose the tunnle service you want to use
# Available options: ngrok, playit
tunnel_service = "argo"  # Si prefieres NGROK cambialo aqui <---
print("Usando", tunnel_service)
    

if tunnel_service == "ngrok":
  !pip -q install pyngrok
  from pyngrok import conf, ngrok
  
  # Ask for the ngrok authtoken
  print("Consigue tu authtoken de https://dashboard.ngrok.com/auth")
  import getpass


  # V---------------------------- T O K E N------------------------------------------V


  authtoken = ""     # <---- TOKEN NGROK (Pon aqui tu Token[ TIENE QUE ESTAR ENTRA LAS "COMILLAS"])

  ! ngrok authtoken $authtoken # login to ngrok

  # Sets default ngrok region

  # V--------------REGIONES DISPONIBLES---------------V
  # ap - Asia/Pacific
  # au - Australia
  # eu - Europa
  # in - India
  # jp - Japon
  # sa - America del Sur
  # us - Estados Unidos
  conf.get_default().region = 'us'  # <---- Cambia esto por la region que quieras (SOLO NGROK)

  # Connect to ngrok
  url = ngrok.connect(25565, 'tcp')
  print('La IP de tu Servidor es: ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))


elif tunnel_service == "playit":
  ! curl -SsL https://playit-cloud.github.io/ppa/key.gpg | sudo apt-key add -
  ! sudo curl -SsL -o /etc/apt/sources.list.d/playit-cloud.list https://playit-cloud.github.io/ppa/playit-cloud.list
  print('INICIANDO SERVIDOR ...')
  ! sudo apt update &>/dev/null && sudo apt install playit &>/dev/null && echo "Playit.gg instalado" || echo "Error en instalar Playit"
  if colabconfig["server_type"] == "forge":
    version = colabconfig["server_version"]
    if colabconfig["server_version"] < "1.17":
      oldpathtoforge = glob.glob(f"/content/drive/My Drive/Minecraft-server/forge-{version}-*.jar")
      if oldpathtoforge:
        path = oldpathtoforge[0]
        print(path)
        ! playit & java $memory_allocation -jar "{path}" nogui
      else:
        print("No forge universal jar found.")
    else:
      pathtoforge = glob.glob(f"/content/drive/My Drive/Minecraft-server/forge-{version}-*/unix_*.txt")
      if pathtoforge:
        path = pathtoforge[0]
        print(path)
        ! playit & java @user_jvm_args.txt "@{path}" "$@"
      else:
        print("No unix_args.txt found")
  else: 
    ! playit & java $memory_allocation $server_flags -jar $jar_name nogui

print('INICIANDO SERVIDOR ...')

if colabconfig["server_type"] == 'forge':
  version = colabconfig["server_version"]
  if colabconfig["server_version"] < "1.17":
    oldpathtoforge = glob.glob(f"/content/drive/My Drive/Minecraft-server/forge-{version}-*.jar")
    if oldpathtoforge:
      path = oldpathtoforge[0]
      print(path)
      ! playit & java $memory_allocation -jar "{path}" nogui
    else:
      print("No forge universal jar found.")
  else:
    pathtoforge = glob.glob(f"/content/drive/My Drive/Minecraft-server/forge-{version}-*/unix_*.txt")
    if pathtoforge:
      path = pathtoforge[0]
      print(path)
      ! playit & java @user_jvm_args.txt "@{path}" "$@"
    else:
      print("No unix_args.txt found")
else: 
  ! java $memory_allocation $server_flags -jar $jar_name nogui



# CREAR SERVIDOR


The code below will download a server for you and accept the EULA. After running these scripts, your server will be ready to run.

**Download the Minecraft server**

The code below will download Paper, a high-performance fork of the Vanilla server.
Other server platforms can be used by placing the server.jar in the Drive folder manually.

In [ ]:
# Replace "1.19.2" with your desired server version.
# Available versions:
# - 1.19.2
# - 1.19.1
# - 1.19
# - 1.18.2
# - 1.18.1
# - 1.18
# - 1.17.1
# - 1.17
# - 1.16.4
# - 1.16.3
# - 1.15.2
# - 1.14.4
# - 1.13.2
# - 1.12.2
# - 1.11.2
# - 1.10.2
# - 1.9.4
# - 1.8.8
# Newer versions might work too, however this isn't guaranteed.
version = '1.12.2'

#forge_version = "43.2.8"  # If you are going to use forge 

# Elige un Servidor --> versiones disponibles: fabric, paper, forge, snapshot, vanilla
server_type = 'forge'

from google.colab import drive
import requests
import json


drive.mount('/content/drive')

! mkdir "/content/drive/My Drive/Minecraft-server"
%cd "/content/drive/My Drive/Minecraft-server"

if server_type == "paper":
  a = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version)
  b = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]))

  print("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"])

  ServerURL = "https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"]

 

if server_type == "fabric":
  ServerURL = "https://maven.fabricmc.net/net/fabricmc/fabric-installer/0.11.2/fabric-installer-0.11.2.jar"

if server_type == "vanilla":
  ServerURL = "https://serverjars.com/api/fetchJar/vanilla/vanilla/" + version

if server_type == "snapshot":
  ServerURL = "https://serverjars.com/api/fetchJar/vanilla/snapshot/" + version
  
if server_type == "forge":
  ServerURL = "https://serverjars.com/api/fetchJar/modded/forge/" + version
  

jar_name = {'paper': 'server.jar', 'fabric': 'fabric-installer.jar', 'forge': 'forge.jar', 'vanilla': 'vanilla.jar', 'snapshot': 'snapshot.jar'}

print('Descargando a Google Drive...')

r = requests.get(ServerURL)

if r.status_code == 200:
  with open('/content/drive/My Drive/Minecraft-server/' + jar_name[server_type], 'wb') as f:
    f.write(r.content)
else:
  print('Error '+ str(r.status_code) + 'La version que escogiste probablemente no funcion / Most likely you entered a unsupported version. Try running the code again if you think that shouldn\'t have happened.')

# Running specific install path.
if server_type == "fabric":
  !java -jar fabric-installer.jar server -mcversion $version -downloadMinecraft

if server_type == "forge":
  %cd "/content/drive/My Drive/Minecraft-server"
  !java -jar forge.jar --installServer

colabconfig = {"server_type": server_type,
               "server_version": version}

json.dump(colabconfig, open("colabconfig.json",'w'))

print('Completado!')


# Please read the file stored in your server folder before running this command. 
# Also, go to https://www.minecraft.net/en-us/eula to read Minecraft's EULA.

# Make sure Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/Minecraft-server"
!echo "eula=true" >> eula.txt
